In [1]:
# !pip install Sastrawi
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# nltk.download('stopwords')
# nltk.download('punkt')

In [2]:
data = pd.read_excel('Scraping Gabung.xlsx')
data.to_numpy()
data

,Username,Komen
0,DiagaPaiker,"@Muhdzahir @hrdbacot Biar mudah di pahami, mir..."
1,chalistamall,#Jubahoperasi Jubah operasi\n#Apdgown Apd Gow...
2,romiaann,Tapi kok dokter2/tenaga medis gak jelasin ya? ...
3,sundarkeu_,"Gimana ya, para dokter atau tenaga medis lain ..."
4,aburnin6hill,@mediocrickey semua itu ga bakal terjadi kalo ...
...,...,...
1941,jojo_duren88,d jktim ad ga min
1942,MS4wLjABAAAAjUt-zinMzURP7MM9i1QRJM2fJIpJb0yQ8m...,di jakarta ada ga pak
1943,_semesta04,Lampung menunggu...kapan suhu...😳😳
1944,callme123lisa,Lok mana si


In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

# ----- START Setup Stopword and Additional Stopword
stopwords            = set(stopwords.words('indonesian'))
additional_stopwords = ['g', 'yg', 'tdk', 'gak', 'x', 'nya', 'rt', 'jg', 'kkb', 'sbg', 'bgt', 'kek', 'tu', 'gw', 'ngga', 'nih', 'deh', 'it', 'tp', 'ank', 'sy', 'dh','duhh','ko','jd','klo','lahh','in','amin','q','x','smg',
                        'huh', 'klu','oo', 'aaa', 'aaaaaaa', 'aamiiiin']
new_stopwords        = stopwords.union(additional_stopwords)
# ----- END

# ----- START setup stemmer kata
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# ----- END

positive_data = pd.read_csv('corpus/positive.txt', delimiter = "\t") # load kamus kata positive bisa di tambahkan di file positive.txt
negative_data = pd.read_csv('corpus/negative.txt', delimiter = "\t") # load kamus kata negative bisa di tambahkan di file negative.txt

def caseFolding(text):
    text = str(text).lower()
    text = ' '.join(re.sub("[@#][A-Za-z0-9_]+|([^0-9A-Za-z \t])|(\w+:\/\/\S+)+|<.*?>|^https?:\/\/.*[\r\n]*"," ", text).split(' '))
    text = re.sub(r"\d+", "", text)
    return text

def tokenizing(text):
    text = nltk.tokenize.word_tokenize(text)
    return text

def filtering(token):
    filter_ = [t for t in token if t not in new_stopwords]
    return " ".join(filter_)

def stemming(filtered_token):
    stemmed = stemmer.stem(filtered_token)
    return stemmed
 
def check_positive_negative(stemming_words):
    words = stemming_words.split() # Split kata - kata dari hasil steming
    positive = 0 # Deklarasi positive 0
    negative = 0 # Deklarasi Negative 0
    for word in words:
        m_positive = 1 if len([n for n in positive_data.to_numpy() if n == word]) > 0 else 0 # Check apakah kata positive atau negative
        m_negative = 1 if len([n for n in negative_data.to_numpy() if n == word]) > 0 else 0 # Check apakah kata pisitive atau negative
        positive += m_positive # Jika positive tambahkan ke count positive
        negative += m_negative # jika negative tambahkan ke count negative
        # Jika data tidak positive ataupun negative di skip
    hasil = ""
    if(positive > negative):
        hasil = "Positive" #Jika hasil positive lebih banyak dari hasil negative
    elif(positive == negative):
        hasil = "Neutral"
    else:   
        hasil = "Negative" #Jika hasil negative lebih banyak dari hasil positive
    return hasil

In [6]:
dataset     = list()
total_label = {
    'Negative':0,
    'Positive':0,
    'Neutral':0
}

for index, row in data.iterrows():
    print('Proccessing Index '+str(index)+'... ', end = '')
    tweet_text   = row['Komen']
    _casefolding = caseFolding(tweet_text)
    _tokenizing  = tokenizing(_casefolding)
    _filtering   = filtering(_tokenizing)
    if len(_filtering) > 3: # jika hasil filtering lebih dari 3 huruf maka akan di lanjutkan
        _stemming    = stemming(_filtering)
        _cek_posneg  = check_positive_negative(_stemming)

        label = 0
        if _cek_posneg == 'Positive':
            label = 1
            total_label['Positive']+=1
        elif _cek_posneg == 'Neutral':
            label = 0
            total_label['Neutral']+=1
        else:
            label = -1
            total_label['Negative']+=1

        print('Done...')
        dataset.append({
            "user"  : row['Username'],
            "tweet" : tweet_text,
            "stem"  : _stemming,
            "label" : label,
            "tokenizing" : _tokenizing
        })
    else:
        print('Length not enough ', len(_filtering))

Proccessing Index 0... Done...
Proccessing Index 1... Done...
Proccessing Index 2... Done...
Proccessing Index 3... Done...
Proccessing Index 4... Done...
Proccessing Index 5... Done...
Proccessing Index 6... Done...
Proccessing Index 7... Done...
Proccessing Index 8... Done...
Proccessing Index 9... Done...
Proccessing Index 10... Done...
Proccessing Index 11... Done...
Proccessing Index 12... Done...
Proccessing Index 13... Done...
Proccessing Index 14... Done...
Proccessing Index 15... Done...
Proccessing Index 16... Done...
Proccessing Index 17... Done...
Proccessing Index 18... Done...
Proccessing Index 19... Done...
Proccessing Index 20... Done...
Proccessing Index 21... Done...
Proccessing Index 22... Done...
Proccessing Index 23... Done...
Proccessing Index 24... Done...
Proccessing Index 25... Done...
Proccessing Index 26... Done...
Proccessing Index 27... Done...
Proccessing Index 28... Done...
Proccessing Index 29... Done...
Proccessing Index 30... Done...
Proccessing Index 

Proccessing Index 253... Done...
Proccessing Index 254... Done...
Proccessing Index 255... Done...
Proccessing Index 256... Done...
Proccessing Index 257... Done...
Proccessing Index 258... Done...
Proccessing Index 259... Done...
Proccessing Index 260... Done...
Proccessing Index 261... Done...
Proccessing Index 262... Done...
Proccessing Index 263... Done...
Proccessing Index 264... Done...
Proccessing Index 265... Done...
Proccessing Index 266... Done...
Proccessing Index 267... Done...
Proccessing Index 268... Done...
Proccessing Index 269... Done...
Proccessing Index 270... Done...
Proccessing Index 271... Done...
Proccessing Index 272... Done...
Proccessing Index 273... Done...
Proccessing Index 274... Done...
Proccessing Index 275... Done...
Proccessing Index 276... Done...
Proccessing Index 277... Done...
Proccessing Index 278... Done...
Proccessing Index 279... Done...
Proccessing Index 280... Done...
Proccessing Index 281... Done...
Proccessing Index 282... Done...
Proccessin

Proccessing Index 500... Done...
Proccessing Index 501... Done...
Proccessing Index 502... Done...
Proccessing Index 503... Done...
Proccessing Index 504... Done...
Proccessing Index 505... Done...
Proccessing Index 506... Done...
Proccessing Index 507... Done...
Proccessing Index 508... Done...
Proccessing Index 509... Done...
Proccessing Index 510... Done...
Proccessing Index 511... Done...
Proccessing Index 512... Done...
Proccessing Index 513... Done...
Proccessing Index 514... Done...
Proccessing Index 515... Done...
Proccessing Index 516... Done...
Proccessing Index 517... Done...
Proccessing Index 518... Done...
Proccessing Index 519... Done...
Proccessing Index 520... Done...
Proccessing Index 521... Done...
Proccessing Index 522... Done...
Proccessing Index 523... Done...
Proccessing Index 524... Done...
Proccessing Index 525... Done...
Proccessing Index 526... Done...
Proccessing Index 527... Done...
Proccessing Index 528... Done...
Proccessing Index 529... Done...
Proccessin

Proccessing Index 750... Done...
Proccessing Index 751... Done...
Proccessing Index 752... Done...
Proccessing Index 753... Done...
Proccessing Index 754... Done...
Proccessing Index 755... Done...
Proccessing Index 756... Done...
Proccessing Index 757... Done...
Proccessing Index 758... Done...
Proccessing Index 759... Done...
Proccessing Index 760... Done...
Proccessing Index 761... Done...
Proccessing Index 762... Done...
Proccessing Index 763... Done...
Proccessing Index 764... Done...
Proccessing Index 765... Done...
Proccessing Index 766... Done...
Proccessing Index 767... Done...
Proccessing Index 768... Done...
Proccessing Index 769... Done...
Proccessing Index 770... Done...
Proccessing Index 771... Done...
Proccessing Index 772... Done...
Proccessing Index 773... Done...
Proccessing Index 774... Done...
Proccessing Index 775... Done...
Proccessing Index 776... Done...
Proccessing Index 777... Done...
Proccessing Index 778... Done...
Proccessing Index 779... Done...
Proccessin

Proccessing Index 998... Done...
Proccessing Index 999... Done...
Proccessing Index 1000... Done...
Proccessing Index 1001... Done...
Proccessing Index 1002... Done...
Proccessing Index 1003... Done...
Proccessing Index 1004... Done...
Proccessing Index 1005... Done...
Proccessing Index 1006... Done...
Proccessing Index 1007... Done...
Proccessing Index 1008... Done...
Proccessing Index 1009... Done...
Proccessing Index 1010... Done...
Proccessing Index 1011... Done...
Proccessing Index 1012... Done...
Proccessing Index 1013... Done...
Proccessing Index 1014... Done...
Proccessing Index 1015... Done...
Proccessing Index 1016... Done...
Proccessing Index 1017... Done...
Proccessing Index 1018... Done...
Proccessing Index 1019... Done...
Proccessing Index 1020... Done...
Proccessing Index 1021... Done...
Proccessing Index 1022... Done...
Proccessing Index 1023... Done...
Proccessing Index 1024... Done...
Proccessing Index 1025... Done...
Proccessing Index 1026... Done...
Proccessing Inde

Proccessing Index 1243... Done...
Proccessing Index 1244... Done...
Proccessing Index 1245... Length not enough  2
Proccessing Index 1246... Length not enough  2
Proccessing Index 1247... Length not enough  2
Proccessing Index 1248... Done...
Proccessing Index 1249... Length not enough  2
Proccessing Index 1250... Length not enough  3
Proccessing Index 1251... Done...
Proccessing Index 1252... Done...
Proccessing Index 1253... Length not enough  3
Proccessing Index 1254... Done...
Proccessing Index 1255... Done...
Proccessing Index 1256... Done...
Proccessing Index 1257... Done...
Proccessing Index 1258... Done...
Proccessing Index 1259... Done...
Proccessing Index 1260... Done...
Proccessing Index 1261... Done...
Proccessing Index 1262... Done...
Proccessing Index 1263... Done...
Proccessing Index 1264... Done...
Proccessing Index 1265... Done...
Proccessing Index 1266... Done...
Proccessing Index 1267... Done...
Proccessing Index 1268... Done...
Proccessing Index 1269... Done...
Proc

Proccessing Index 1466... Done...
Proccessing Index 1467... Done...
Proccessing Index 1468... Done...
Proccessing Index 1469... Done...
Proccessing Index 1470... Done...
Proccessing Index 1471... Done...
Proccessing Index 1472... Done...
Proccessing Index 1473... Done...
Proccessing Index 1474... Done...
Proccessing Index 1475... Done...
Proccessing Index 1476... Done...
Proccessing Index 1477... Done...
Proccessing Index 1478... Done...
Proccessing Index 1479... Done...
Proccessing Index 1480... Done...
Proccessing Index 1481... Done...
Proccessing Index 1482... Done...
Proccessing Index 1483... Done...
Proccessing Index 1484... Done...
Proccessing Index 1485... Done...
Proccessing Index 1486... Length not enough  3
Proccessing Index 1487... Done...
Proccessing Index 1488... Done...
Proccessing Index 1489... Done...
Proccessing Index 1490... Length not enough  3
Proccessing Index 1491... Done...
Proccessing Index 1492... Length not enough  3
Proccessing Index 1493... Length not enough

Proccessing Index 1702... Done...
Proccessing Index 1703... Done...
Proccessing Index 1704... Done...
Proccessing Index 1705... Done...
Proccessing Index 1706... Done...
Proccessing Index 1707... Done...
Proccessing Index 1708... Done...
Proccessing Index 1709... Done...
Proccessing Index 1710... Done...
Proccessing Index 1711... Done...
Proccessing Index 1712... Done...
Proccessing Index 1713... Done...
Proccessing Index 1714... Done...
Proccessing Index 1715... Done...
Proccessing Index 1716... Done...
Proccessing Index 1717... Done...
Proccessing Index 1718... Done...
Proccessing Index 1719... Done...
Proccessing Index 1720... Done...
Proccessing Index 1721... Done...
Proccessing Index 1722... Done...
Proccessing Index 1723... Done...
Proccessing Index 1724... Done...
Proccessing Index 1725... Done...
Proccessing Index 1726... Done...
Proccessing Index 1727... Done...
Proccessing Index 1728... Done...
Proccessing Index 1729... Done...
Proccessing Index 1730... Done...
Proccessing In

Proccessing Index 1940... Done...
Proccessing Index 1941... Done...
Proccessing Index 1942... Done...
Proccessing Index 1943... Done...
Proccessing Index 1944... Done...
Proccessing Index 1945... Done...


In [7]:
df1 = pd.DataFrame(dataset)
df1.to_csv('dataset_labeling_finish.csv')
df1.head()

,user,tweet,stem,label,tokenizing
0,DiagaPaiker,"@Muhdzahir @hrdbacot Biar mudah di pahami, mir...",biar mudah paham tenaga sehat pns dokter pppk ...,1,"[biar, mudah, di, pahami, mirip, seperti, tena..."
1,chalistamall,#Jubahoperasi Jubah operasi\n#Apdgown Apd Gow...,jubah operasi apd gown surgical gown jubah ope...,-1,"[jubah, operasi, apd, gown, surgical, gown, ju..."
2,romiaann,Tapi kok dokter2/tenaga medis gak jelasin ya? ...,dokter tenaga medis jelasin ya orang kritis hm...,-1,"[tapi, kok, dokter, tenaga, medis, gak, jelasi..."
3,sundarkeu_,"Gimana ya, para dokter atau tenaga medis lain ...",gimana ya dokter tenaga medis tuh ngasih tau p...,-1,"[gimana, ya, para, dokter, atau, tenaga, medis..."
4,aburnin6hill,@mediocrickey semua itu ga bakal terjadi kalo ...,ga kalo indonesia bijak safe abortions kaya uk...,1,"[semua, itu, ga, bakal, terjadi, kalo, indones..."


In [8]:
print('Total Label Dari Semua data :')
print(total_label)

Total Label Dari Semua data :
{'Negative': 515, 'Positive': 407, 'Neutral': 934}


In [9]:
#menampilkan kata yang muncul dari terbesar ke terkecil
from collections import Counter

count = Counter(" ".join(df1.stem).split())
df1 = pd.DataFrame.from_dict(count, orient='index', columns=['Jumlah']).reset_index()
df1.sort_values(['Jumlah'], ascending=[False])

,index,Jumlah
10,medis,367
3,tenaga,350
24,ya,221
167,anak,210
160,aja,201
...,...,...
2119,ng,1
2120,sgitu,1
2124,sumbat,1
2125,cair,1
